In [1]:
!pip install langchain langchain_community langchain_openai wikipedia langchain_experimental langsmith pandas langgraph tavily-python gigachat gradio

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━

In [2]:
import functools
import operator
from pprint import pprint
from pydantic import BaseModel, Field
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
import gradio as gr
import os
import requests
import logging
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv
from IPython.display import Image, display
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory, ConversationSummaryMemory, VectorStoreRetrieverMemory
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.chains import ConversationChain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.schema import Document
from langchain.retrievers import TFIDFRetriever
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.embeddings import GigaChatEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentExecutor, create_tool_calling_agent, create_react_agent
from langchain.tools.retriever import create_retriever_tool
from langchain.chat_models.gigachat import GigaChat
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.tools import tool
from langchain_community.docstore import InMemoryDocstore
from langchain.prompts import PromptTemplate
from langchain.document_loaders import DirectoryLoader
from langchain.chains import RetrievalQA

In [3]:
load_dotenv('.env')

True

# **Код Чекалиной Нины**

In [4]:
BASE_API_URL = "https://gitlab.com/api/v4/projects/yandexdataschool%2Fml-handbook/repository/tree"
RAW_URL_TEMPLATE = "https://gitlab.com/yandexdataschool/ml-handbook/-/raw/master/{path}"
REF = "master"
OUTPUT_DIR = "ml-handbook"


os.makedirs(OUTPUT_DIR, exist_ok=True)


def download_md_file(file_path):
    raw_url = RAW_URL_TEMPLATE.format(path=file_path)
    response = requests.get(raw_url)
    if response.status_code == 200:
        # Создаем подкаталоги в зависимости от пути файла
        relative_dir = os.path.dirname(file_path)  # Получаем относительный путь директории
        save_dir = os.path.join(OUTPUT_DIR, relative_dir)  # Создаем путь внутри `ml-handbook`
        os.makedirs(save_dir, exist_ok=True)  # Убедимся, что директория существует

        file_name = os.path.join(save_dir, os.path.basename(file_path))  # Полный путь до файла
        with open(file_name, "w", encoding="utf-8") as file:
            file.write(response.text)
        print(f"Скачан файл: {file_name}")
    else:
        print(f"Не удалось скачать файл: {file_path} (код {response.status_code})")


def process_folder(folder_path):
    print(f"Обрабатываем папку: {folder_path}")
    response = requests.get(BASE_API_URL, params={"ref": REF, "path": folder_path})
    if response.status_code == 200:
        items = response.json()
        for item in items:
            item_path = item["path"]
            if item["type"] == "tree":  # Если это вложенная папка
                process_folder(item_path)
            elif item["type"] == "blob" and item_path.endswith(".md"):  # Если это файл .md
                download_md_file(item_path)
    else:
        print(f"Ошибка API при обработке папки {folder_path}: {response.status_code}")


folders = [
    "chapters/clustering",
    "chapters/cross_validation",
    "chapters/decision_tree",
    "chapters/ensembles",
    "chapters/grad_boost",
    "chapters/hyperparameters_tuning",
    "chapters/intro",
    "chapters/linear_models",
    "chapters/matrix_diff",
    "chapters/metric_based",
    "chapters/ml_theory",
    "chapters/model_evaluation",
    "chapters/neural_nets",
    "chapters/optimization",
    "chapters/prob_bayes",
    "chapters/prob_calibration",
    "chapters/prob_genclass",
    "chapters/prob_glm",
    "chapters/prob_intro",
    "chapters/prob_maxent",
]

for folder in folders:
    process_folder(folder)


Обрабатываем папку: chapters/clustering
Обрабатываем папку: chapters/clustering/images
Скачан файл: ml-handbook/chapters/clustering/intro.md
Обрабатываем папку: chapters/cross_validation
Обрабатываем папку: chapters/cross_validation/images
Скачан файл: ml-handbook/chapters/cross_validation/intro.md
Обрабатываем папку: chapters/decision_tree
Обрабатываем папку: chapters/decision_tree/images
Скачан файл: ml-handbook/chapters/decision_tree/intro.md
Обрабатываем папку: chapters/ensembles
Обрабатываем папку: chapters/ensembles/images
Скачан файл: ml-handbook/chapters/ensembles/intro.md
Обрабатываем папку: chapters/grad_boost
Обрабатываем папку: chapters/grad_boost/images
Скачан файл: ml-handbook/chapters/grad_boost/intro.md
Обрабатываем папку: chapters/hyperparameters_tuning
Обрабатываем папку: chapters/hyperparameters_tuning/images
Скачан файл: ml-handbook/chapters/hyperparameters_tuning/intro.md
Обрабатываем папку: chapters/intro
Обрабатываем папку: chapters/intro/images
Скачан файл: ml-h

# **Код Алхименкова Леонида** *Курсив*

In [17]:
def load_md_files(directory):
    documents = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(".md"):
                file_path = os.path.join(root, file)
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()
                    documents.append(Document(page_content=content, metadata={"path": file_path}))
    return documents

OUTPUT_DIR = 'ml-handbook'
documents = load_md_files(OUTPUT_DIR)
# print(f"Загружено {len(documents)} документов.")

if not documents:
    raise ValueError("Документы не загружены. Проверьте путь к директории и содержимое файлов.")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=500)
texts = text_splitter.split_documents(documents)

retriever = TFIDFRetriever.from_documents(texts)

In [18]:
tool = create_retriever_tool(
    retriever,
    "text_retriever_for_ML",
    "Use this tool to extract relevant machine learning information."
)
tools = [tool]

tool_strings = "\n".join([f"{tool.name}: {tool.description}" for tool in tools])

prompt_template = PromptTemplate(
    input_variables=["query", "agent_scratchpad", "tool_names", "tools"],
    template=(
    "You are an assistant that provides relevant information only on machine learning in documents. "
    "When you are asked a question, you must clearly state your arguments in the same language in which the question is being asked. "
    "Use the following format for your output:\n\n"
    "Thought: [Your thought process]\n"
    "Action: [The name of the tool to use]\n"
    "Action Input: [The input to the tool]\n"
    "Observation: [The result from the tool]\n\n"
    "Final Answer: [Your final response to the query]\n\n"
    "Available tools: {tool_names}\n"
    "{tools}\n"
    "Query: {query}\n"
    "{agent_scratchpad}"
)

)

giga_key = os.environ.get("SB_AUTH_DATA")
giga = GigaChat(credentials=giga_key, model="GigaChat", timeout=30, verify_ssl_certs=False)

retriever_agent = create_react_agent(giga, tools, prompt_template)

def remove_duplicates(documents):
    seen_paths = set()
    unique_docs = []
    for doc in documents:
        if doc.metadata['path'] not in seen_paths:
            unique_docs.append(doc)
            seen_paths.add(doc.metadata['path'])
    return unique_docs

# **Код Веснина Константина**

In [19]:
def get_answer(test_query):
    result = retriever_agent.invoke({
        "query": test_query,
        "agent_scratchpad": "",
        "tool_names": ", ".join([tool.name for tool in tools]),
        "tools": tool_strings,
        "intermediate_steps": []
    })
    output = ''
    output += result.log + '\n'

    # try:
    #     retrieved_docs = retriever.get_relevant_documents(test_query)
    #     unique_docs = remove_duplicates(retrieved_docs)
    #     output += f"\n\nТакже дополнительно можно ознакомиться с информацией по следующим {len(unique_docs)}-м уникальных сслыках\n"
    #     for doc in unique_docs:
    #         doc.metadata['path'] = doc.metadata['path'].replace('ml-handbook/chapters/', '')
    #         output += f"ссылка: https://gitlab.com/yandexdataschool/ml-handbook/-/blob/master/chapters/{doc.metadata['path']}?ref_type=heads\n"
    # except AttributeError as e:
    #     output += f"Ошибка: {e}"

    return output


with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            gen_button = gr.Button('Сгенерировать ответ')
            question = gr.Textbox(label='Ваш вопрос')
            answer = gr.Textbox(label='Ответ')

    gen_button.click(
        fn=get_answer, inputs=[question], outputs=[answer]
    )

demo.queue().launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ee3348a3436ecced77.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
